# **TAREA 2 - MACHINE LEARNING 2**
### **Assignment 2: Principal Component Analysis, Neural Networks**

**Integrantes:**

* Llaro Castro, Diego Renato
* Rios Meza, Jennifer Saskia
* Vargas Flores, Johanna Antonella

## **Part I: Eigenfaces for Face Recognition**

In [1]:
# IMPORTAMOS LIBRERÍAS
import os
import numpy as np
import imageio
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.cm as cm

In [6]:
# RUTA BASE DE IMÁGENES
base_path = '/workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images'

# LEEMOS TRAIN Y TEST CSV/TXT
train_csv = pd.read_csv(
    '/workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/train.txt',
    sep=" ",
    header=None,
    names=["image", "label"]
)
test_csv = pd.read_csv(
    '/workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/test.txt',
    sep=" ",
    header=None,
    names=["image", "label"]
)


In [7]:
# CARGAMOS DATOS DE ENTRENAMIENTO
train_data, train_labels = [], []
for idx, row in train_csv.iterrows():
    img_name = row['image']
    label = int(row['label'])
    img_path = os.path.join(base_path, img_name)

    if not os.path.exists(img_path):
        print(f"Archivo no encontrado: {img_path}")
        continue

    im = imageio.v2.imread(img_path)
    train_data.append(im.reshape(2500,))
    train_labels.append(label)

train_data = np.array(train_data, dtype=float)
train_labels = np.array(train_labels, dtype=int)


Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_63.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_14.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_09.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_05.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_01.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_53.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_34.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_62.png
Archivo no encontrado: /workspaces/TAREA2_ML2_LLARO_RIOS_VARGAS/data/images/./faces/images/person01_12.png
Archivo no encontrado: /workspaces/TA

In [11]:
# MOSTRAMOS UNA IMAGEN DE EJEMPLO
plt.imshow(train_data[0].reshape(50,50), cmap=cm.Greys_r)
plt.title("IMAGEN DE ENTRENAMIENTO EJEMPLO")
plt.show()

IndexError: index 0 is out of bounds for axis 0 with size 0